In [ ]:
from gurobipy import *
import numpy as np
import math
from sklearn.gaussian_process.kernels import RBF
from scipy.optimize import minimize
import time
import scipy

import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoLarsCV
import statistics
import time
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils.random import sample_without_replacement

t_1 = time.time()
#parameters
X_multiplier=7#multiplier of X in demand calculation
priceMultiplier=1#multiplier of price in kernel
Gamma=3
P_low=Gamma
P_high=Gamma+7

sigma=1#bandwidth of kernel
dim=1#dim of feature X

def policy_to_evaluate(x,p):
    return 1.1*p

def expected_demand(x_arr,p_arr):
    #return (-p_arr+P_high)/(P_high-P_low)
    p_arr=np.minimum(P_high,p_arr)
    p_arr=np.maximum(P_low,p_arr)
    return (x_arr*X_multiplier-p_arr+P_high)/(P_high-P_low+X_multiplier)

def realized_demand(x_arr,p_arr,rndseed):
    np.random.seed(rndseed+52028)
    return np.random.binomial(1,expected_demand(x_arr,p_arr))
    
#generate training dataset
rndseed=2352#927#2352
train_size=30#100
np.random.seed(rndseed+883)
X_train=np.random.uniform(0,1,train_size)
#np.random.seed(rndseed+5325)
#P_train=np.random.uniform(P_low,P_high,train_size)
P_train=P_high-(P_high-P_low)*X_train#deterministic pricing
D_train=realized_demand(X_train,P_train,rndseed+917)

#generate testing dataset
P_test=np.zeros(train_size)
for i in range(0,train_size):
    P_test[i]=policy_to_evaluate(X_train[i],P_train[i])

#pre-compute values needed
R_train=np.multiply(D_train,P_train)
expected_R_train=np.multiply(P_train,expected_demand(X_train,P_train))
expected_R_test=np.multiply(P_test,expected_demand(X_train,P_test))
expected_val_test=sum(expected_R_test)/train_size

#for computing kernel only: normalize prices, and multiply by given factor
P_train_norm=priceMultiplier*(P_train-P_low)/ (P_high-P_low)

Z_train=np.zeros((train_size,dim+1))
for i in range(0,train_size):
    Z_train[i]=np.append(X_train[i],P_train_norm[i])#historical policy
Y_train=np.zeros((train_size,dim+1))
for i in range(0,train_size):
    Y_train[i]=np.append(X_train[i],policy_to_evaluate(X_train[i],P_train_norm[i]))#new policy

#gram matrix
kernel = RBF(sigma)
Z_and_Y=np.zeros((train_size*2,dim+1))
Z_and_Y[0:train_size,:]=Z_train
Z_and_Y[train_size:2*train_size,:]=Y_train
G=kernel(Z_and_Y)
G=G+0.0001*np.identity(train_size*2)#make G positive semi-definite
C=np.linalg.cholesky(G*2)#C is lower triangular, CC^\top=G

#preparation
rowSumHalfG=np.transpose(np.sum(G[:,train_size:2*train_size],axis=1).reshape(-1,1))#sum each row of right half of G
D3=np.transpose(rowSumHalfG).dot(rowSumHalfG)/(train_size**2)#the third term in Dw
arr_m=np.zeros((train_size,train_size*2,train_size*2))
for i in range(0,train_size):
    temp2=G[:,i].reshape(-1,1)
    arr_m[i]=temp2.dot(np.transpose(temp2))

In [ ]:
#choose penalty paramter for Nathan's method
#input training features and realized revenue. output choice of parameter lambda
def nathal_choose_param(Z_train,R_train): 
    #gram matrix
    kernel = RBF(sigma)
    K=kernel(Z_train)#computing kernel takes <1 sec
    R_normalize=R_train-np.average(R_train)
    #evals,evecs=np.linalg.eig(K)#this method is slow
    evals=scipy.linalg.eigh(K, eigvals_only=True)#this method assumes real symmetric matrix, and is fast

    #compute for each lamda
    lamda_list=np.arange(1,51)/2#when using lambda<1, determinant of A will be zero, causing exp(det(A)) to return error
    neg_log_likelihood_list=np.zeros(len(lamda_list))

    for i in range(0,len(lamda_list)):
        lamda=lamda_list[i]
        A=K+lamda*np.identity(train_size)
        b=np.linalg.solve(A, R_normalize)#solve the linear system instead of inverting A, takes 3-4 secs
        #A_det=np.prod(lamda+evals)
        log_det_A=sum(np.log(lamda+evals))
        neg_log_likelihood_list[i]=np.transpose(R_normalize)@b+0.5*log_det_A
        #no need to include the last term in log-likelihood, which is constant (0.5*train_size*math.log(2*math.pi))

    optimal_lamda=lamda_list[np.argmin(neg_log_likelihood_list)]
    
    return optimal_lamda

In [ ]:
#Nathan's method
#input penalty parameter lambda. Output weights
def nathans_method(lamda): 
    G_aug=np.copy(G)
    for i in range(0,train_size):
        G_aug[i][i]=G_aug[i][i]+lamda**2#add variance term

    #evals and evectors of G
    evals,evecs=scipy.linalg.eigh(G_aug, eigvals_only=False)#this method assumes real symmetric matrix, and is fast
    A=np.diag(evals)
    c=np.sum(evecs[0:train_size,0:2*train_size],axis=0)

    #build optimization model
    m = Model()
    m.Params.LogToConsole = 0#suppress Gurobipy printing
    # Add variables
    v=m.addMVar ( train_size*2, ub=1.0,lb=-1.0,name="v" )
    # Set objective function
    #m.Params.OptimalityTol=0.01
    m.setObjective(v@A@v, GRB.MINIMIZE) 
    # Add constraints
    m.addConstr(v@c==1)
    m.addConstr(evecs[train_size:2*train_size,0:2*train_size]@v==-np.ones(train_size)/train_size)
    #m.addConstr(S[0:train_size,0:2*train_size]@v>=-np.zeros(train_size))#w_i>=0
    #m.params.method=0#0 is primal simplex, 1 is dual simplex, 2 is barrier
    m.update()
    m.optimize()

    #get weights
    v_star=np.zeros(2*train_size)
    for i in range(0,2*train_size):
        v_star[i]=v[i].x
    w=evecs[0:train_size,0:2*train_size]@v_star
    
    return w

In [ ]:
#subroutine that computes phi(w) using Ben-tal's method
#input: starting weights. output: phi(w) and gradient
def subroutine(weights,Gamma):

    #compute D(w)
    wSumHalfG=(G[:,0:train_size].dot(weights)).reshape(-1,1)
    D2=2*wSumHalfG.dot(rowSumHalfG)/train_size#the second term in Dw
    D1=wSumHalfG.dot(np.transpose(wSumHalfG))#the first term in Dw
    for i in range(0,train_size):
        D1=D1-arr_m[i]*weights[i]**2
    Dw=-2*D1+D2+np.transpose(D2)-2*D3#Dw may not be PSD

    #compute S
    M=np.linalg.solve(C,Dw)#solve matrix M for CM=Dw
    B=np.linalg.solve(C,np.transpose(M))#solve matrix B for CB=M^\top

    evals,evecs=scipy.linalg.eigh(B, eigvals_only=False)#symmetric QR
    Q=np.copy(evecs)
    delta=np.copy(evals)
    S=np.linalg.solve(np.transpose(C),Q)
    
    #compute bw, epsilon
    temp2=np.multiply(np.multiply(weights,weights),P_train)
    bw=-G[:,0:train_size]@temp2
    epsilon=np.transpose(S).dot(bw)
    
    #build optimization model
    m = Model()
    m.Params.LogToConsole = 0#suppress Gurobipy printing
    # Add variables
    y=m.addMVar ( train_size*2, ub=100.0,lb=-100.0,name="y" )
    x=m.addMVar ( train_size*2, ub=100.0,lb=-100.0,name="x" )    
    # Set objective function
    #m.Params.OptimalityTol=0.01
    m.setObjective(epsilon@x+delta@y, GRB.MINIMIZE) 
    # Add constraints
    m.addConstr(np.ones(train_size*2)@y<=Gamma**2)
    m.addConstrs(x[i]@x[i]*0.5<=y[i] for i in range(train_size*2))
    #m.params.method=0#0 is primal simplex, 1 is dual simplex, 2 is barrier
    #m.params.NonConvex=2
    m.update()
    m.optimize()
    
    #get key values
    obj = m.getObjective()
    obj_val = -obj.getValue() #return phi(w) 
    
    x_star=np.zeros(2*train_size)
    for i in range(0,2*train_size):
        x_star[i]=x[i].x
    q=S@x_star#worst case r()
    #print('worst case revenue function r',q)
    
    #compute gradient
    grad=np.zeros(train_size)
    for i in range(0,train_size):
        temp1=np.transpose(G[:,i]).reshape(-1,1)
        temp2=np.transpose(wSumHalfG)-weights[i]*np.transpose(temp1)-rowSumHalfG/train_size
        Hessian=2*temp1.dot(temp2)
        grad[i]=q.dot(Hessian.dot(np.transpose(q)))
        grad[i]=grad[i]+2*weights[i]*P_train[i]*np.transpose(G[:,i]).dot(q)
    
    return obj_val,grad

In [ ]:
#main loop. Frank-Wolfe algorithm 
#compute weights that minimize MSE(w) 
#use weights from nathan's method as starting point 
def FW(w,Gamma):

    ini_obj,grad=subroutine(w,Gamma) 
    k=1 
    L=Gamma*50#a guess on lipschitz constant of phi(w) 
    print('initial objective value: ',ini_obj) 
    maxIter=1000
    MSE_arr=np.zeros(maxIter+1) 
    MSE_arr[0]=ini_obj 
    stepSize_arr=np.zeros(maxIter+1) 
    step_size=1
    gt=0.1
    while (gt>=0.02 and k<=maxIter):

        #compute descent direction
        obj_val,grad=subroutine(w,Gamma)
        ind=np.argmin(grad)
        s_t=np.zeros(train_size)
        s_t[ind]=1
        descent_dir=s_t-w

        #compute step size
        #step_size=2/(k+100)#default stepsize of frank-wolfe
        gt=-grad@descent_dir
        step_size=min(gt/(L*descent_dir@descent_dir),1)

        #store key outputs
        MSE_arr[k]=obj_val
        stepSize_arr[k]=step_size

        #update weights
        w=w+step_size*descent_dir
        if (k % 20==0):
            print('iter: ',k,'obj val: ',obj_val,'gt: ',gt,'coord: ',ind)
        k=k+1
        
        #if gradients are equa, then KKT conditions are satisfied
        #max(grad)-min(grad)>=0.01
        
    #truncate dummy values
    total_iter=k
    MSE_arr=MSE_arr[0:k]
    stepSize_arr=stepSize_arr[0:k]
    
    return w,MSE_arr

In [ ]:
#compute true MSE, use our formula for MSE
def true_MSE(w):
    bias=w@expected_R_train-expected_val_test
    #temp1=np.multiply(R_train,P_train-R_train)#this will be zeros since R_train is either P_train or zero
    temp1=np.multiply(expected_R_train,P_train-expected_R_train)
    temp2=np.multiply(w,w)
    variance=temp1@temp2
    #variance=expected_R_test@(P_test-expected_R_test)/(train_size**2)
    return bias**2+variance

In [ ]:
#main loop
#test our method for oracle Gamma
w_0=nathans_method(1)#starting point for our weights
#Gamma_list=np.arange(1,11)*0.3#first trial
#Gamma_list=3+np.arange(1,11)*0.2#2nd trial
Gamma_list=np.arange(1,11)*0.3
true_MSE_ours=np.zeros(len(Gamma_list))
worst_MSE_ours=np.zeros(len(Gamma_list))
est_val_ours=np.zeros(len(Gamma_list))
w_ours_all=np.zeros((len(Gamma_list),train_size))
w_ours=np.copy(w_0)
for l in range(0,len(Gamma_list)):
    Gamma=Gamma_list[l]
    w_ours,MSE_arr=FW(w_ours,Gamma)#warm start at previous soln
    w_ours_all[l]=w_ours#store outputs
    true_MSE_ours[l]=true_MSE(w_ours)
    worst_MSE_ours[l]=MSE_arr[-1]
    est_val_ours[l]=w_ours@R_train
    print('Gamma= ',Gamma,' complete')

In [ ]:
#test nathan's method for oracle lambda
#assume Gamma=3
#lamda_list=np.arange(1,11)#first trial
#lamda_list=np.arange(1,11)/10#2nd trial
#lamda_list=np.arange(1,11)*0.01#where we find oracle lambda
lamda_list=np.arange(1,101)*0.01
true_MSE_nathan=np.zeros(len(lamda_list))
worst_MSE_nathan=np.zeros(len(lamda_list))
est_val_nathan=np.zeros(len(lamda_list))
w_nathan_all=np.zeros((len(lamda_list),train_size))
for l in range(0,len(lamda_list)):
    lamda=lamda_list[l]
    w_nathan=nathans_method(lamda)
    w_nathan_all[l]=w_nathan
    true_MSE_nathan[l]=true_MSE(w_nathan)
    worst_MSE_nathan[l],_=subroutine(w_nathan,3)
    est_val_nathan[l]=w_nathan@R_train
    if l%10==0:
        print('lambda= ',lamda,' complete')

    
t_2 = time.time()
print('run time: ',t_2-t_1)

In [ ]:
#plot our MSE against gamma
from matplotlib import pyplot
pyplot.plot(Gamma_list, true_MSE_ours,label='true MSE')
pyplot.plot(Gamma_list, worst_MSE_ours,label='worst case MSE')
plt.scatter(Gamma_list[np.argmin(true_MSE_ours)],min(true_MSE_ours),s=20,color="blue",label='min true MSE')
pyplot.title('our method MSE')
pyplot.xlabel('Gamma')
pyplot.ylabel('')
pyplot.legend(bbox_to_anchor = (1.0, 1), loc = 'upper center')
pyplot.show()

In [ ]:
#plot nathans MSE against gamma
from matplotlib import pyplot
pyplot.plot(lamda_list, true_MSE_nathan,label='true MSE')
pyplot.plot(lamda_list, worst_MSE_nathan,label='worst case MSE')
plt.scatter(lamda_list[np.argmin(true_MSE_nathan)],min(true_MSE_nathan),s=20,color="blue",label='min true MSE')
pyplot.title('Nathans method MSE')
pyplot.xlabel('lambda')
pyplot.ylabel('')
pyplot.legend(bbox_to_anchor = (1.0, 1), loc = 'upper center')
pyplot.show()

In [ ]:
#plot our est val against gamma
delta=0.05
from matplotlib import pyplot
lb_ours=np.zeros(len(Gamma_list))
for i in range(0,len(Gamma_list)):
    lb_ours[i]=est_val_ours[i]-(worst_MSE_ours[i]/delta)**0.5
pyplot.plot(Gamma_list, est_val_ours,label='estimated value')
plt.axhline(y = expected_val_test, color = 'r', linestyle = '-',label='expected value')
pyplot.plot(Gamma_list, lb_ours,label='lower bound')
pyplot.title('Our method estimated value')
pyplot.xlabel('Gamma')
pyplot.ylabel('')
pyplot.legend(bbox_to_anchor = (1.0, 1), loc = 'upper center')
pyplot.show()

In [ ]:
#plot nathans est val against lambda
from matplotlib import pyplot
lb_nathan=np.zeros(len(lamda_list))
for i in range(0,len(lamda_list)):
    lb_nathan[i]=est_val_nathan[i]-(worst_MSE_nathan[i]/delta)**0.5
pyplot.plot(lamda_list, est_val_nathan,label='estimated value')
plt.axhline(y = expected_val_test, color = 'r', linestyle = '-',label='expected value')
pyplot.plot(lamda_list, lb_nathan,label='lower bound')
pyplot.title('Nathans method estimated value')
pyplot.xlabel('lambda')
pyplot.ylabel('')
pyplot.legend(bbox_to_anchor = (1.0, 1), loc = 'upper center')
pyplot.show()

In [ ]:
#now plot our weights for different Gamma
from matplotlib import pyplot
for l in range(len(Gamma_list)):
    if l%2==0:
        plt.scatter(X_train, w_ours_all[l],alpha=0.5,label='Gamma= '+str(Gamma_list[l]))
pyplot.title('our weights')
pyplot.xlabel('feature X')
pyplot.ylabel('weights')
pyplot.legend(bbox_to_anchor = (1.0, 1), loc = 'upper center')
pyplot.show()

In [ ]:
#now plot nathans weights for different lambda
from matplotlib import pyplot
for l in range(len(lamda_list)):
    if l%20==0:
        plt.scatter(X_train, w_nathan_all[l],alpha=0.5,label='lambda= '+str(lamda_list[l]))
pyplot.title('Nathans weights')
pyplot.xlabel('feature X')
pyplot.ylabel('weights')
pyplot.legend(bbox_to_anchor = (1.0, 1), loc = 'upper center')
pyplot.show()

In [ ]:
#plot nathans weights for larger lambda
#compute weights
lamda_list2=np.arange(1,11)
w_nathan_all2=np.zeros((len(lamda_list2),train_size))
for l in range(0,len(lamda_list2)):
    w_nathan_all2[l]=nathans_method(lamda_list2[l])

#plot weights
from matplotlib import pyplot
for l in range(len(lamda_list2)):
    if l%2==0:
        plt.scatter(X_train, w_nathan_all2[l],alpha=0.5,label='lambda= '+str(lamda_list2[l]))
pyplot.title('Nathans weights')
pyplot.xlabel('feature X')
pyplot.ylabel('weights')
pyplot.legend(bbox_to_anchor = (1.0, 1), loc = 'upper center')
pyplot.show()